In [ ]:
# demo.ipynb
!git clone https://github.com/madelezhia/image-classifier-app.git
#%cd image-classifier-app

!pip install -q torch torchvision flask pillow pyngrok > /dev/null

# lance Flask en arrière-plan
get_ipython().system_raw('python server.py &')

# ==========================
# 3. Tunnel ngrok + Secret
# ==========================
from pyngrok import ngrok, conf
import time, requests, os
from google.colab import userdata   # accès aux Secrets

ngrok.kill()

try:
    TOKEN = userdata.get("NGROK")   # Secret nommé "NGROK"
except Exception:
    TOKEN = ""                      # pas de Secret → gratuit limité

if TOKEN:
    conf.get_default().auth_token = TOKEN

time.sleep(3)                       # laisse Flask démarrer

import socket, time
for _ in range(10):          # 10 essais
    try:
        socket.create_connection(("127.0.0.1", 5000), timeout=1)
        print("✔ Flask écoute sur 5000")
        break
    except Exception as e:
        print("⏳ attente Flask…", e); time.sleep(2)
else:
    print("❌ Flask introuvable"); raise RuntimeError("Flask ne s’est pas lancé")

http_tunnel = ngrok.connect(5000, "http")
public_url = http_tunnel.public_url
print("🌍 URL publique de la démo : ", public_url)

# ==========================
# 4. Test rapide
# ==========================
image_path = "tests/pineapple-sits-table-with-other-fruits_726745-5475-3767101720.jpg"
with open(image_path, "rb") as f:
    r = requests.post(f"{public_url}/predictImage", files={"image": (os.path.basename(image_path), f, "image/jpeg")})


print("Status :", r.status_code)
print("Headers :", r.headers)
print("Text brut (200 car) :", r.text[:200])

if r.status_code == 200:
    print("Réponse : ", r.json())
else:
    print("Error Response:", r.text)

print('📂  Image de test : tests/pineapple-sits-table-with-other-fruits_726745-5475-3767101720.jpg')
